## Workign in cells in nbclassic (Jupyter 6)

In [ ]:
#| default_exp utils.nbclassic

In [ ]:
#| export
from IPython import get_ipython
from IPython.display import display, clear_output, Markdown, Javascript
import json

In [ ]:

#| exporti
from time import sleep
def inject_js(js:str, delay=0):
    """Inject some javascript into the notebook and clear the output to prevent it from running on reload"""

    display_handle = display(Javascript(js), display_id=True)
    # If we keep the outpur, it will be run on every notebook load.
    display_handle.update(Javascript(""))

In [ ]:
#| export

def patch_kernel():
    payload = """
    console.log("patching nbclassic execute function...")
    Jupyter.CodeCell.prototype.execute = function (stop_on_error) {
        if (!this.kernel) {
            console.log(i18n.msg._("Can't execute cell since kernel is not set."));
            return;
        }

        if (stop_on_error === undefined) {
            if (this.metadata !== undefined &&
                    this.metadata.tags !== undefined) {
                if (this.metadata.tags.indexOf('raises-exception') !== -1) {
                    stop_on_error = false;
                } else {
                    stop_on_error = true;
                }
            } else {
               stop_on_error = true;
            }
        }

        this.clear_output(false, true);
        var old_msg_id = this.last_msg_id;
        if (old_msg_id) {
            this.kernel.clear_callbacks_for_msg(old_msg_id);
            delete Jupyter.CodeCell.msg_cells[old_msg_id];
            this.last_msg_id = null;
        }
        if (this.get_text().trim().length === 0) {
            // nothing to do
            this.set_input_prompt(null);
            return;
        }
        this.set_input_prompt('*');
        this.element.addClass("running");
        var callbacks = this.get_callbacks();


        const cell_index = Jupyter.notebook.find_cell_index(this)

        let extras = {
            cell_index : cell_index,
            cell_id: this.id
        }
        let text = this.get_text().trim()
        let firstLine = text.split('\\n')[0];

        // Parse the magic command
        if (firstLine.startsWith("%%fr")) {
            // Separate args by spaces or tabs
            let parts = firstLine.split(/\\s+|\\t+/);
            let magic = parts[0];
            if (parts.length > 1) {
                let magic_args = parts.slice(1);
                let plusNArg = magic_args.find(arg => arg.startsWith('+') && !isNaN(parseInt(arg.slice(1))));
                if (plusNArg) {
                    let n = parseInt(plusNArg.slice(1));
                    let start_pos = Math.max(0, cell_index - n);
                    cells = Jupyter.notebook.get_cells().slice(start_pos, cell_index);

                    extras = {
                        cells_above: cells,
                        ...extras
                    }
                }
            }
        }

        this.last_msg_id = this.kernel.execute(
            this.get_text(),
            callbacks,
            {silent: false, store_history: true, stop_on_error : stop_on_error, ...extras });
        Jupyter.CodeCell.msg_cells[this.last_msg_id] = this;
        this.render();
        this.events.trigger('execute.CodeCell', {cell: this});
        var that = this;
        function handleFinished(evt, data) {
            if (that.kernel.id === data.kernel.id && that.last_msg_id === data.msg_id) {
                    that.events.trigger('finished_execute.CodeCell', {cell: that});
                that.events.off('finished_iopub.Kernel', handleFinished);
              }
        }
        this.events.on('finished_iopub.Kernel', handleFinished);
    };
    Jupyter.notebook.events.trigger('set_dirty.Notebook', {value: true});
    console.log("Done.")
    """
    inject_js(payload)

In [ ]:
#| export
def add_cell(
        idx:int = None, # Index of the cell to add. If none, add the cell under the selected one.
        cell_type:str = "code" # Type of cell to add. Can be "code", "markdown", "raw"
    ):
    """
    Add a new notebook cell.
    """
    if not idx:
        index_payload = "let index = Jupyter.notebook.get_selected_index()+1;"
    else:
        index_payload = f"let index = {idx}"

    payload = f"""
    {index_payload}

    Jupyter.notebook.insert_cell_at_index("{cell_type}", index)
    let cell = Jupyter.notebook.get_cell(index);
    cell.events.trigger('set_dirty.Notebook', {{value: true}});
    """

    inject_js(payload)

In [ ]:
#| export
def update_cell(
    idx:int, # Index of the cell to update. None to update the current cell
    text:str, # Text to set in the cell
    flush:bool = True # Notify Jupyter that the cell has been updated.
    ):

    def escape_for_js(text):
        # Use json.dumps to escape the string for JavaScript
        escaped = json.dumps(text)
        # Remove the surrounding quotes added by json.dumps
        escaped = escaped[1:-1]
        # Escape backticks and ${} sequences
        return escaped.replace('`', '\\`').replace('${', '\\${')

    payload = f"""
    let cell = Jupyter.notebook.get_cell({idx})
    cell.set_text(`{escape_for_js(text)}`)
    """
    if flush:
         payload = payload + "\nJupyter.notebook.events.trigger('set_dirty.Notebook', {value: true});"
    inject_js(payload)

In [ ]:
#| export
def execute_cell(
        idx:int # Index of the cell to execute. They start at 0
    ):
    payload = f"""
    console.log("execute_cell", {idx});
    Jupyter.notebook.events.trigger('set_dirty.Notebook', {{value: true}});
    let cell = Jupyter.notebook.get_cell({idx})
    cell.execute()
    """
    # tt = display(f"About to run the cell {idx}...", display_id=True)
    inject_js(payload)

In [ ]:
#| export
def render_cell(idx:int): # Cell to render.
    """Render a markdown cell"""

    payload = f"""
    let cell = Jupyter.notebook.get_cell({idx})
    cell.unrender()
    Jupyter.notebook.events.trigger('set_dirty.Notebook', {{value: true}});

    cell.render()

    """

    inject_js(payload)

In [ ]:
#| export
def delete_cell(idx:int): # Cell to delete.
    """Delete a cell"""

    payload = f"""
    console.log("deleting cell", {idx});
    Jupyter.notebook.delete_cell({idx});
    Jupyter.notebook.events.trigger('set_dirty.Notebook', {{value: true}});
    """

    inject_js(payload)

In [ ]:
#| export
def get_index():
    ip = get_ipython()
    return ip.parent_header["content"].get("cell_index", -1)